<a href="https://www.kaggle.com/code/rubanzasilva/fastai-neural-network-gradient-boosting?scriptVersionId=188940590" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Binary Classification of Insurance Cross 

The objective of this competition is to predict which customers respond positively to an automobile insurance offer.

A company that has been offering health insurance to a segment of customers wants to find out whether their health insurance customers are willing to also get/sign up for their motor vehicle insurance plan 

## Library Imports

In [12]:
%pip install catboost
%pip install optuna
%pip install optuna_distributed
%pip install openfe
%pip install seaborn
%pip install xgboost
%pip install lightgbm
%pip install fastkaggle
%pip install h2o
%pip install -Uqq fastbook
%pip install -q -U autogluon.tabular
%pip install autogluon
#ip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.3 MB/s eta 0:00:00
  Attempting uninstall: distributed
    Found existing installation: distributed 2024.1.1
    Uninstalling distributed-2024.1.1:
      Successfully uninstalled distributed-2024.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
rapids-dask-dependency 24.4.1a0 requires dask==2024.1.1, but you have dask 2024.5.2 which is incompatible.
rapids-dask-dependency 24.4.1a0 requires dask-expr==0.4.0, but you have dask-expr 1.1.2 which is incompatible.
rapids-dask-dependency 24.4.1a0 requires distributed==2024.1.1, but you have distributed 2024.5.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart 

In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e7/sample_submission.csv
/kaggle/input/playground-series-s4e7/train.csv
/kaggle/input/playground-series-s4e7/test.csv


In [14]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [15]:
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.tabular.all import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random
from tqdm import tqdm
from ipywidgets import interact

from fastai.imports import *
np.set_printoptions(linewidth=130)


from pathlib import Path
import os


from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import roc_auc_score,accuracy_score,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score



#transformers and pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn import set_config

import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier

import lightgbm as lgb
from lightgbm import LGBMClassifier

from catboost import CatBoostClassifier,CatBoostRegressor,Pool, metrics, cv

import optuna
from optuna.samplers import TPESampler
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice
from optuna.samplers import TPESampler
import warnings


matplotlib.rc('image', cmap='Greys')

from fastkaggle import setup_comp



from openfe import OpenFE, transform
from autogluon.tabular import TabularDataset, TabularPredictor

#from IPython.display import FileLink

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [16]:
!ls /kaggle/input/

playground-series-s4e7


In [17]:
path = Path('/kaggle/input/playground-series-s4e7/')
path

Path('/kaggle/input/playground-series-s4e7')

In [18]:
train_df = pd.read_csv(path/'train.csv',index_col='id')
test_df = pd.read_csv(path/'test.csv',index_col='id')
sub_df = pd.read_csv(path/'sample_submission.csv')
#original_df = pd.read_csv('/kaggle/input/academic-success-dataset/data.csv',delimiter=';')

In [19]:
train_df.shape,test_df.shape

((11504798, 11), (7669866, 10))

## Subset

In [ ]:
train_subset = train_df.sample(n=1000000,replace=False)
test_subset = test_df.sample(n=500000,replace=False)

In [ ]:
train_subset.shape,test_subset.shape

In [ ]:
#cont_names,cat_names = cont_cat_split(train_subset, dep_var='Response')
#splits = RandomSplitter(valid_pct=0.2)(range_of(train_subset))
#to = TabularPandas(train_subset, procs=[Categorify, FillMissing,Normalize],
#                  cat_names = cat_names,
#                  cont_names = cont_names,
#                  y_names='Response',
#                  y_block=CategoryBlock(),
#                  splits=splits)
#dls = to.dataloaders(bs=64)
#test_dl = dls.test_dl(test_subset)
#X_train, y_train = to.train.xs, to.train.ys.values.ravel()
#X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

# Full Dataset

Below i use the fastai cont_cat_split which returns the values of a particular dataframe as either categorical or continous values depending on the cardinality of the column values which is how many levels a particular column has.

This function takes an argument of max_card whose default is 20. If the number of unique values is above the max_card, then that variables is considered to be continuos and vice versa.

In [20]:
cont_names,cat_names = cont_cat_split(train_df, dep_var='Response')
len(cat_names),len(cont_names)

(5, 5)

In [21]:
cont_names

['Age', 'Region_Code', 'Annual_Premium', 'Policy_Sales_Channel', 'Vintage']

In [22]:
cat_names

['Gender',
 'Driving_License',
 'Previously_Insured',
 'Vehicle_Age',
 'Vehicle_Damage']

Below, i use fastai's RandomSplitter to divide my dataset into a train and validation set, in this example it creates a validation set with 20% of the training dataset.


In [23]:
splits = RandomSplitter(valid_pct=0.2)(range_of(train_df))

Below i build a TabularPandas Object which is a subclass of the Tabular class that has methods we can use to handle tabular data preparation and preprocessing.

This object takes in arguments where i define my training dataset and preprocessing steps i.e. Categorify, FillMissing,Normalize.

We also define our categorical and continuous variables resulting from the cont_cat_split.
Furthermore, i define my Target variable, the y which in this case is Response, y_block represents the type of problem.

Lastly we have the splits argument which was defined above and represents the dataset split.

In [24]:
to = TabularPandas(train_df, procs=[Categorify, FillMissing,Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names='Response',
                   y_block=CategoryBlock(),
                   splits=splits)

In [25]:
to.xs.iloc[:2]

,Gender,Driving_License,Previously_Insured,Vehicle_Age,Vehicle_Damage,Age,Region_Code,Annual_Premium,Policy_Sales_Channel,Vintage
id,,,,,,,,,,
3321034,2,2,1,2,2,-0.825867,-1.417947,0.130413,0.732322,-0.786311
8422000,1,2,1,2,2,-0.492411,-1.802805,-0.511315,0.732322,-0.786311


In [26]:
dls = to.dataloaders(bs=64)
test_dl = dls.test_dl(test_df)

In [27]:
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

In [28]:
dls.show_batch()

,Gender,Driving_License,Previously_Insured,Vehicle_Age,Vehicle_Damage,Age,Region_Code,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,Female,1,1,< 1 Year,No,24.000000,46.999999,25651.999781,151.999998,12.999999,0
1,Male,1,0,< 1 Year,Yes,29.000000,28.000000,55289.999728,26.000001,41.999996,1
2,Female,1,1,< 1 Year,No,22.000001,46.000000,25899.999969,159.999999,160.000000,0
3,Female,1,1,1-2 Year,No,51.000000,28.000000,24793.000049,26.000001,177.000000,0
4,Male,1,1,< 1 Year,No,27.000000,29.000000,2629.999079,151.999998,173.000000,0
5,Male,1,0,1-2 Year,Yes,44.000000,32.000000,2629.999079,124.000000,298.000001,0
6,Female,1,1,< 1 Year,No,23.000001,8.000000,49181.000899,151.999998,179.000000,0
7,Male,1,0,1-2 Year,Yes,44.000000,39.000000,2629.999079,26.000001,274.999998,0
8,Female,1,1,< 1 Year,No,25.000000,36.000000,2629.999079,151.000001,203.000001,0
9,Male,1,0,1-2 Year,Yes,55.000000,28.000000,32875.999897,26.000001,76.000001,0


# Neural Network

In [ ]:
learn = tabular_learner(dls, metrics=RocAucBinary())

In [ ]:
learn.lr_find(suggest_funcs=(slide,valley))

In [ ]:
%%time
learn.fit_one_cycle(8,0.005)


In [ ]:
dl = test_dl
nn_preds = learn.get_preds(dl=dl)
nn_preds_x = learn.get_preds()[0]
a_preds, _ = learn.get_preds(dl=dl)
nn_preds_y = a_preds.squeeze(1)
nn_preds_proba = (a_preds[:, 1])

In [ ]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['Response'] = nn_preds_proba
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
sub

# Neural Network Ensemble

In [ ]:
def ensemble():
    learn = tabular_learner(dls, metrics=RocAucMulti())
    with learn.no_bar(),learn.no_logging(): learn.fit(5, 0.005)
    return learn.get_preds(test_dl=test_dl)[0]

In [ ]:
learns = [ensemble() for _ in range(5)]

In [ ]:
ens_preds = torch.stack(learns).mean(0)

# Random Forest

In [ ]:
%%time
rf = RandomForestClassifier(100, min_samples_leaf=3)
rf_model = rf.fit(X_train, y_train);

rf_preds = tensor(rf_model.predict(test_dl.xs))

rf_preds_x = tensor(rf_model.predict(X_test))

#mse = mean_absolute_error(y_test, rf_preds_x)
#rmse = np.sqrt(mse)

#accuracy_score(y_test,rf_preds_x)
roc_auc_score(y_test,rf_preds_x)

In [ ]:
#RandomForestClassifier??

# Gradient Boosting

# XGBoost

In [ ]:
xgb_optuna_params = {
    'n_estimators': 929,
    'alpha': 2.287466581490129e-05,
    'subsample': 0.8766675651018592,
    'colsample_bytree': 0.288332829334817,
    'max_depth': 8,
    'min_child_weight': 6,
    'learning_rate': 0.024083411832750343,
    'gamma': 0.001816649055813574
}

In [ ]:
print(xgb.get_config())

In [ ]:
xgb_params = {
    'colsample_bylevel': 0.21416299403295808,
    'colsample_bynode': 0.6208356380953189,
    'colsample_bytree': 0.19219680400212635,
    'gamma': 0.6051664515971382,
    'max_bin': 682,
    'max_delta_step': 5.264818337431145,
    'max_depth': 68,
    'min_child_weight': 5.23408291542125,
    'n_estimators': 10000,
    'n_jobs': -1,
    'objective': "binary:logistic",
    'reg_alpha': 0.46516016901463414,
    'reg_lambda': 0.8410553418334902,
    'subsample': 0.802533192662325,
    'verbosity': 0,
    'eval_metric': "auc",
    #'early_stopping_rounds': 200,
    'tree_method': 'gpu_hist'  # Use GPU acceleration
}

In [ ]:
%%time
xgb_model = xgb.XGBClassifier(**xgb_params)
xgb_model = xgb_model.fit(X_train, y_train)

xgb_preds = tensor(xgb_model.predict(test_dl.xs))
xgb_preds_proba = tensor(xgb_model.predict_proba(test_dl.xs))[:, 1]

xgb_preds_x = tensor(xgb_model.predict(X_test))
xgb_preds_x_proba = tensor(xgb_model.predict_proba(X_test))[:, 1]

xgb_score = roc_auc_score(y_test,xgb_preds_x_proba)
xgb_score

In [ ]:
!ls

In [ ]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['Response'] = xgb_preds_proba
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
sub

### Cross validation implementation

In [ ]:
#scores = cross_val_score(xgb_model,X_train, y_train,cv=5,scoring='roc_auc')
#scores.mean()

In [ ]:
%%time
from sklearn.model_selection import KFold
import numpy as np
from sklearn.metrics import roc_auc_score

K_FOLDS = 5  # Number of folds for cross-validation
kfold = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

# This part is just to demonstrate how the indices work
for train_index, test_index in kfold.split(train_df):
    print(f'train: {len(train_index)} samples, test: {len(test_index)} samples')

fold_scores = []

for train_index, val_index in kfold.split(X_train):
    X_fold_train, X_fold_val = X_train.iloc[train_index], X_train.iloc[val_index]
    y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]
    
    # Train the model on the current fold
    Xgb_model_fold = xgb.XGBClassifier(**xgb_params)
    Xgb_model_fold.fit(X_fold_train, y_fold_train)
    
    # Predict probabilities on the validation set for the current fold
    y_pred_fold = Xgb_model_fold.predict_proba(X_fold_val)[:, 1]
    y_pred_fold_tt = Xgb_model_fold.predict_proba(test_dl.xs)[:, 1]
    
    # Calculate and store the AUC-ROC score for the current fold
    score = roc_auc_score(y_fold_val, y_pred_fold)
    fold_scores.append(score)

# Calculate the average score across all folds
avg_score = np.mean(fold_scores)
print(f"Average AUC-ROC Score Across All Folds: {avg_score}")

In [ ]:
y_pred_fold_tt.shape

In [ ]:
avg_score

In [ ]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['Response'] = y_pred_fold_tt
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
sub

In [ ]:
xgb_model = xgb.XGBClassifier(alpha=0.2271025051979102,
                    gamma=0.07819541270693704,
                    eta=0.294535700691525,
                    max_depth=15,
                    max_bin=174857,
                    min_child_weight=1,
                    max_child_weight=10,
                    n_estimators=9500,
                    subsample=0.8362024536878123,
                    tree_method="hist",
                    device="cuda",
                    use_label_encoder=False,
                    enable_categorical=True,
                    eval_metric='auc',
                    objective='binary:logistic',
                    colsample_bytree=0.42784607351667425)
xgb_model = xgb_model.fit(X_train, y_train)

xgb_preds = tensor(xgb_model.predict(test_dl.xs))

xgb_preds_x = tensor(xgb_model.predict(X_test))

#accuracy_score(y_test,xgb_preds_x)
xgb_score = roc_auc_score(y_test,xgb_preds_x)
xgb_score

# LightGBM

In [ ]:
lgbm_params = {'n_estimators': 1190, 'learning_rate': 0.22952000374471332, 'max_depth': 13, 'reg_alpha': 8.200152384535924,
          'reg_lambda': 4.285393733702208, 
          'num_leaves': 100, 'subsample': 0.6497981764924947, 'colsample_bytree': 0.37368304607248115,\
               'device_type':'gpu'}

In [ ]:
%%time
#ds subset
lgbm_model = lgb.LGBMClassifier(**lgbm_params)
lgbm_model = lgbm_model.fit(X_train, y_train)

#test set preds
lgbm_preds = tensor(lgbm_model.predict(test_dl.xs))
lgbm_preds_prob = tensor(lgbm_model.predict_proba(test_dl.xs))
lgbm_positive_preds = (lgbm_preds_prob[:, 1])

#validation set preds
lgbm_preds_x = tensor(lgbm_model.predict(X_test))
lgbm_preds_x_prob = tensor(lgbm_model.predict_proba(X_test))
lgbm_positive_preds_x = (lgbm_preds_x_prob[:, 1])

lgbm_score = roc_auc_score(y_test,lgbm_positive_preds_x)
lgbm_score

#lgb_preds_x_prob = tensor(lgb_model.predict_proba(X_test))

lgbm_score_prob = roc_auc_score(y_test,lgbm_positive_preds_x)
lgbm_score_prob


[LightGBM] [Info] Number of positive: 1131646, number of negative: 8072193
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.201688 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 723
[LightGBM] [Info] Number of data points in the train set: 9203839, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.122954 -> initscore=-1.964752
[LightGBM] [Info] Start training from score -1.964752
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

In [ ]:
lgbm_score_prob

## Hyperparamter optimization with Optuna

In [ ]:
# finetuned optuna code
%%time
import optuna
from optuna.samplers import TPESampler
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
import numpy as np

def objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 300, 1200),
        'subsample_for_bin': trial.suggest_int('subsample_for_bin', 20000, 300000),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 500),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-9, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-9, 10.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'subsample': trial.suggest_float('subsample', 0.25, 1.0),
        'max_depth': trial.suggest_int('max_depth', 1, 15),
        'device_type': 'gpu'
    }
    
    K_FOLDS = 5  # Number of folds for cross-validation
    kfold = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

    fold_scores = []
    
    #for train_index, test_index in kfold.split(train_df):
    #print(f'train: {len(train_index)} samples, test: {len(test_index)} samples')

    for train_index, val_index in kfold.split(X_train):
        X_fold_train, X_fold_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[val_index]
        
        # Train the model on the current fold
        lgb_model_fold = lgb.LGBMClassifier(**params)  # Use params, not lgbm_params
        lgb_model_fold.fit(X_fold_train, y_fold_train)
        
        # Predict probabilities on the validation set for the current fold
        y_pred_fold = lgb_model_fold.predict_proba(X_fold_val)[:, 1]
        
        # Calculate and store the AUC-ROC score for the current fold
        score = roc_auc_score(y_fold_val, y_pred_fold)
        fold_scores.append(score)
    
    # Calculate the average score across all folds
    return np.mean(fold_scores)

# Create and run the study
study = optuna.create_study(sampler=TPESampler(n_startup_trials=30, multivariate=True, seed=0), direction="maximize")
study.optimize(objective, n_trials=100)

print('Best value:', study.best_value)
print('Best trial:', study.best_trial.params)

In [39]:
print('Best value:', study.best_value)
print('Best trial:', study.best_trial.params)

Best value: 0.880810142621536
Best trial: {'num_leaves': 320, 'learning_rate': 0.13981961408994045, 'n_estimators': 843, 'subsample_for_bin': 172567, 'min_child_samples': 223, 'reg_alpha': 0.0028770084050677926, 'reg_lambda': 2.3761404778025532e-05, 'colsample_bytree': 0.9350638004692479, 'subsample': 0.9727470703757719, 'max_depth': 6}


In [40]:
lgb_optuna_run2_params  =  {
    'num_leaves': 320, 
    'learning_rate': 0.13981961408994045, 
    'n_estimators': 843,
    'subsample_for_bin': 172567, 
    'min_child_samples': 223,
    'reg_alpha': 0.0028770084050677926,
    'reg_lambda': 2.3761404778025532e-05,
    'colsample_bytree': 0.9350638004692479,
    'subsample': 0.9727470703757719,
    'max_depth': 6,
    'device_type':'gpu'
}

### With Optuna tuned parameters

In [ ]:
%%time
#ds subset
lgbm_model = lgb.LGBMClassifier(**lgbm_params)
lgbm_model = lgbm_model.fit(X_train, y_train)

#test set preds
lgbm_preds = tensor(lgbm_model.predict(test_dl.xs))
lgbm_preds_prob = tensor(lgbm_model.predict_proba(test_dl.xs))
lgbm_positive_preds = (lgbm_preds_prob[:, 1])

#validation set preds
lgbm_preds_x = tensor(lgbm_model.predict(X_test))
lgbm_preds_x_prob = tensor(lgbm_model.predict_proba(X_test))
lgbm_positive_preds_x = (lgbm_preds_x_prob[:, 1])

lgbm_score = roc_auc_score(y_test,lgbm_positive_preds_x)
lgbm_score

#lgb_preds_x_prob = tensor(lgb_model.predict_proba(X_test))

lgbm_score_prob = roc_auc_score(y_test,lgbm_positive_preds_x)
lgbm_score_prob


# CatBoost

In [32]:
cat_optuna_params = {
    
    'colsample_bylevel': 0.6383474716497279,
    'learning_rate': 0.09475494290429642,
    'random_strength': 0.07771221926568195,
    'max_bin': 490,
    'depth': 5,
    'l2_leaf_reg': 5,
    'boosting_type': 'Plain',
    'bootstrap_type': 'Bernoulli',
    'subsample': 0.8429457747642737,
    'task_type': 'GPU'
    
}

In [33]:
%%time
cat_model = CatBoostClassifier(**cat_optuna_params)
cat_model = cat_model.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=False)

#test set preds
cat_preds = tensor(cat_model.predict(test_dl.xs))


#cat_preds_final = cat_preds.squeeze(1)

#validation set preds
cat_preds_x = tensor(cat_model.predict(X_test))

#cat_preds_x_final = cat_preds_x.squeeze(1)

#accuracy_score(y_test,cat_preds_x)

cat_score = roc_auc_score(y_test,cat_preds_x)
cat_score

CatBoostError: /src/catboost/catboost/private/libs/options/catboost_options.cpp:636: Error: rsm on GPU is supported for pairwise modes only

# Model Predictions

In [ ]:
model_preds = {
    "random forests":roc_auc_score(y_test,rf_preds_x),
    "cat boost":cat_score,
    "lgbm":lgb_score,
    "xgboost":roc_auc_score(y_test,xgb_preds_x),   
}

#model_preds_a = model_preds.sort()
print(model_preds)

In [34]:
cat_preds_x

NameError: name 'cat_preds_x' is not defined

# Model Ensemble

# Submission

In [ ]:
!ls

In [ ]:
submit = pd.read_csv(path/'sample_submission.csv')
submit.Response = xgb_preds
submit.to_csv('submission.csv',index=False)
submit

In [ ]:
!ls

In [ ]:
submit = pd.read_csv(path/'sample_submission.csv')
submit['Response'] = xgb_preds
submit.to_csv('submission.csv', index=False)
sub = pd.read_csv('submission.csv')
sub